In [1]:
# import dependencies 
import os
import gmaps
import pandas as pd

# import Google spi key from Config file
from config import gkey

# configure API key
gmaps.configure(api_key=gkey)

ModuleNotFoundError: No module named 'config'

In [ ]:
# create state with centroid DataFrame
povertyCSV = (os.path.join('povertyRateWithCentroids.csv'))
povertyDF = pd.read_csv(povertyCSV)
# remove index column that imported with CSV
del povertyDF['Unnamed: 0']
# preview poverty DataFrame
povertyDF.head()


In [ ]:
# store latitude and longitude values for state centroids
stateCentroid = povertyDF[['Latitude', 'Longitude']]

# store poverty %s for each state
povertyPercent2012 = povertyDF['2012'].astype(float)
povertyPercent2013 = povertyDF['2013'].astype(float)
povertyPercent2014 = povertyDF['2014'].astype(float)
povertyPercent2015 = povertyDF['2015'].astype(float)
povertyPercent2016 = povertyDF['2016'].astype(float)
povertyPercent2017 = povertyDF['2017'].astype(float)

In [ ]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2012 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2012 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2012.add_layer(heat_layer)

# Display figure
fig2012

In [ ]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2013 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2013 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2013.add_layer(heat_layer)

# Display figure
fig2013

In [ ]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2014 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2014 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2014.add_layer(heat_layer)

# Display figure
fig2014

In [ ]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2015 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2015 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2015.add_layer(heat_layer)

# Display figure
fig2015

In [ ]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2016 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2016 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2016.add_layer(heat_layer)

# Display figure
fig2016

In [ ]:
# set up market locations
markerLocations = povertyDF[['Latitude', 'Longitude']]
# Plot Heatmap
fig2017 = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(stateCentroid, weights=povertyPercent2017 / 2, 
                                 dissipating=False, max_intensity=8,
                                 point_radius=2)

# Add heatmap layer
fig2017.add_layer(heat_layer)

# Display figure
fig2017